In [21]:
from collections import defaultdict

import pandas as pd
import spacy
from textblob import TextBlob

from iliad_network.components.readers import LocalReader

In [22]:
reader = LocalReader()
try:
    poem_text = reader.run()
    clean_text = poem_text.content  # Access the clean text
except ValueError as e:
    print(f"Validation Error: {e}")

In [23]:
# Load spaCy's large English model
nlp = spacy.load("en_core_web_sm")
# python -m spacy download en_core_web_sm


def preprocess_iliad(text):
    # Clean up unnecessary spaces and newlines
    text = text.replace("\n", " ").strip()

    # Tokenize the text using spaCy
    doc = nlp(text)

    # Return the list of sentences and words (tokens)
    sentences = [sent.text for sent in doc.sents]
    words = [token.text for token in doc]

    return sentences, words


# Example usage
iliad_sentences, iliad_words = preprocess_iliad(clean_text)
print(iliad_sentences[:5])  # Preview first 5 sentences

['----------------------------------------------------------------------  BOOK I  Sing, O goddess, the anger of Achilles son of Peleus, that brought countless ills upon the Achaeans.', 'Many a brave soul did it send hurrying down to Hades, and many a hero did it yield a prey to dogs and vultures, for so were the counsels of Jove fulfilled from the day on which the son of Atreus, king of men, and great Achilles, first fell out with one another.   ', 'And which of the gods was it that set them on to quarrel?', 'It was the son of Jove and Leto; for he was angry with the king and sent a pestilence upon the host to plague the people, because the son of Atreus had dishonoured Chryses his priest.', "Now Chryses had come to the ships of the Achaeans to free his daughter, and had brought with him a great ransom: moreover he bore in his hand the sceptre of Apollo wreathed with a suppliant's wreath and he besought the Achaeans, but most of all the two sons of Atreus, who were their chiefs.   "]


In [ ]:
def extract_characters(text):
    doc = nlp(text)
    characters = set()  # Use a set to avoid duplicates

    # Extract named entities (persons) from the text
    for ent in doc.ents:
        if ent.label_ == "PERSON":  # Filter only person entities
            characters.add(ent.text)

    return characters


# Example usage
characters = extract_characters(clean_text)
print(characters)  # Print the set of detected characters

In [ ]:
def build_cooccurrence_matrix(sentences, characters):
    # Create a dictionary to store co-occurrences
    cooccurrence_dict = defaultdict(lambda: defaultdict(int))

    # Loop over sentences
    for sentence in sentences:
        # Extract character mentions in the sentence
        mentioned_characters = set(extract_characters(sentence))

        # If multiple characters are mentioned, increment co-occurrence counts
        for char1 in mentioned_characters:
            for char2 in mentioned_characters:
                if char1 != char2:
                    cooccurrence_dict[char1][char2] += 1

    # Convert the dictionary into a DataFrame for easy viewing
    cooccurrence_df = pd.DataFrame.from_dict(cooccurrence_dict).fillna(0)
    return cooccurrence_df


# Example usage
cooccurrence_matrix = build_cooccurrence_matrix(iliad_sentences, characters)
print(cooccurrence_matrix.head())  # Preview the co-occurrence matrix

            Priam  Jove  Cilla  Chryse  Phoebus Apollo  Calchas  Achilles  \
Jove         23.0   0.0    0.0     0.0             2.0      0.0       2.0   
Cruel Jove    1.0   0.0    0.0     0.0             0.0      0.0       0.0   
Juno          1.0  26.0    0.0     0.0             0.0      0.0       1.0   
Minerva       3.0  36.0    0.0     0.0             0.0      0.0       0.0   
Agamemnon     2.0  11.0    0.0     0.0             0.0      0.0       3.0   

            Idomeneus  Ajax  Chryseis  ...  King Eumelus  Pheres  Antilochus  \
Jove              0.0  10.0       0.0  ...           0.0     0.0         0.0   
Cruel Jove        0.0   0.0       0.0  ...           0.0     0.0         0.0   
Juno              0.0   0.0       0.0  ...           0.0     0.0         0.0   
Minerva           0.0   2.0       0.0  ...           0.0     0.0         0.0   
Agamemnon         0.0   1.0       0.0  ...           0.0     0.0         0.0   

            Epeus  Eueneus  jason  Lycaon  Patroclous  m

In [20]:
def analyze_sentiment(sentence):
    blob = TextBlob(sentence)
    return blob.sentiment.polarity  # Returns sentiment score (-1 to 1)


def sentiment_analysis(sentences, character):
    # Dictionary to store sentiment scores for each character's sentence
    character_sentiments = []

    for sentence in sentences:
        # Check if the character is mentioned in the sentence
        if character in sentence:
            sentiment_score = analyze_sentiment(sentence)
            character_sentiments.append((sentence, sentiment_score))

    return character_sentiments


# Example usage
char = "Achilles"  # Example character
achilles_sentiments = sentiment_analysis(iliad_sentences, char)

# Print a few example sentences with sentiment scores
for sentence, sentiment in achilles_sentiments[:5]:
    print(f"Sentence: {sentence} | Sentiment: {sentiment}")

Sentence: ----------------------------------------------------------------------  BOOK I  Sing, O goddess, the anger of Achilles son of Peleus, that brought countless ills upon the Achaeans. | Sentiment: -0.35
Sentence: Many a brave soul did it send hurrying down to Hades, and many a hero did it yield a prey to dogs and vultures, for so were the counsels of Jove fulfilled from the day on which the son of Atreus, king of men, and great Achilles, first fell out with one another.    | Sentiment: 0.4490740740740741
Sentence: For nine whole days he shot his arrows among the people, but upon the tenth day Achilles called them in assembly- moved thereto by Juno, who saw the Achaeans in their death-throes and had compassion upon them. | Sentiment: 0.2
Sentence: With all sincerity and goodwill he addressed them thus:-   "Achilles, loved of heaven, you bid me tell you about the anger of King Apollo, I will therefore do so; but consider first and swear that you will stand by me heartily in word a